In [25]:
import os
import re, string, unicodedata
import pandas as pd 
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline
from jupyterthemes import jtplot
jtplot.style(theme='solarizedl')

In [26]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer, WhitespaceTokenizer
from nltk.corpus import stopwords
import inflect
from spellchecker import SpellChecker


In [ ]:
import csv

with open('coors.csv', mode='r') as infile:
    reader = csv.reader(infile)
    with open('coors_new.csv', mode='w') as outfile:
    writer = csv.writer(outfile)
    for rows in reader:
        k = rows[0]
        v = rows[1]
        mydict = {k:v for k, v in rows}

In [27]:
from .word_expansion import word_expansion

ImportError: attempted relative import with no known parent package

In [28]:
from setuptools import setup, find_packages 

In [8]:
setup(name = 'word_expansion', packages = find_packages())

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\root\anaconda3\lib\distutils\fancy_getopt.py", line 233, in getopt
    opts, args = getopt.getopt(args, short_opts, self.long_opts)
  File "C:\Users\root\anaconda3\lib\getopt.py", line 95, in getopt
    opts, args = do_shorts(opts, args[0][1:], shortopts, args[1:])
  File "C:\Users\root\anaconda3\lib\getopt.py", line 195, in do_shorts
    if short_has_arg(opt, shortopts):
  File "C:\Users\root\anaconda3\lib\getopt.py", line 211, in short_has_arg
    raise GetoptError(_('option -%s not recognized') % opt, opt)
getopt.GetoptError: option -f not recognized

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\root\anaconda3\lib\distutils\core.py", line 134, in setup
    ok = dist.parse_command_line()
  File "C:\Users\root\anaconda3\lib\distutils\dist.py", line 475, in parse_command_line
    args = parser.getopt(args=self.script_args, object=self)
  File "C:\Users\root\an

TypeError: object of type 'NoneType' has no len()

In [114]:
import sys
sys.path.append('C:\\Users\\root\\Projects\\nlp\\disaster_tweets\\utils')

In [147]:
sys.path.pop()

'C:\\Users\\root\\Projects\\nlp\\disaster_tweets\\utils'

In [29]:
df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [179]:
x = x.replace({r"(^#[A-Z])": r" \1"}, regex=True)

AttributeError: 'list' object has no attribute 'replace'

In [162]:
list1 = []
def camel_case_split(text):
    t = re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', text for text if text.startswith('#')))
    return t
expanded = [camel_case_split(x) for x in df.text]
        
expanded

TypeError: expected string or bytes-like object

In [253]:
na_action='ignore'
df.test = df['text'].apply(re.findall(r"([A-Z])", r'( \1)', x) for x in str(i for i in row.split())], axis=1)
>>> set([re.sub(r"(\W+)$", "", j) for j in set([i for i in text.split() if i.startswith("#")])])

TypeError: unsupported operand type(s) for &: 'str' and 'int'

In [248]:
df

,id,keyword,location,text,target
0,1,NaN,nan,Our Deeds are the Reason of this #earthquake ...,1
1,4,NaN,nan,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,nan,All residents asked to 'shelter in place' are...,1
3,6,NaN,nan,"13,000 people receive #wildfires evacuation o...",1
4,7,NaN,nan,Just got sent this photo from Ruby #Alaska as...,1
...,...,...,...,...,...
7604,10863,NaN,nan,#WorldNews Fallen powerlines on G:link tram: ...,1
7605,10864,NaN,nan,on the flip side I'm at Walmart and there is ...,1
7606,10866,NaN,nan,Suicide bomber kills 15 in Saudi security sit...,1
7608,10869,NaN,nan,Two giant cranes holding a bridge collapse in...,1


In [237]:
df['text'] = " "+df.text
df.drop_duplicates(subset=['text'], inplace=True)

In [45]:

expanded_ht = " ".join([x for x in re.split('([A-Z][a-z]+)', df.text) if x.startswith('#') == True])    
expanded_ht

NameError: name 'i' is not defined

In [238]:
df['location'] = " "+df.location+" "

In [239]:
df.location = df.location.astype(str)

In [180]:
    for row in df.text:
        for x in row:
            x = x.replace({r"(^#[A-Z])": r" \1"}, regex=True)

TypeError: replace() takes no keyword arguments

In [30]:
#### text preprocessing specifically formatted for tweets but will work on any text
def tweet_preprocess(df): 
    """combine regex and nltk processing for tweet text processing"""


    def word_expansion(text):

        word_expansion_dict = {
         'AK': 'Alaska',
         'AL': 'Alabama',
         'AR': 'Arkansas',
         'AZ': 'Arizona',
         'CA': 'California',
         'CO': 'Colorado',
         'CT': 'Connecticut',
         'DC': 'District of Columbia',
         'DE': 'Delaware',
         'FL': 'Florida',
         'GA': 'Georgia',
         'GU': 'Guam',
         'HI': 'Hawaii',
         'IA': 'Iowa',
         'ID': 'Idaho',
         'IL': 'Illinois',
         'IN': 'Indiana',
         'KS': 'Kansas',
         'KY': 'Kentucky',
         'LA': 'Louisiana',
         'MA': 'Massachusetts',
         'MD': 'Maryland',
         'ME': 'Maine',
         'MI': 'Michigan',
         'MN': 'Minnesota',
         'MO': 'Missouri',
         'MP': 'Northern Mariana Islands',
         'MS': 'Mississippi',
         'MT': 'Montana',
         'NC': 'North Carolina',
         'ND': 'North Dakota',
         'NE': 'Nebraska',
         'NH': 'New Hampshire',
         'NJ': 'New Jersey',
         'NM': 'New Mexico',
         'NV': 'Nevada',
         'NY': 'New York',
         'OH': 'Ohio',
         'OK': 'Oklahoma',
         'OR': 'Oregon',
         'PA': 'Pennsylvania',
         'PR': 'Puerto Rico',
         'PW': 'Palau',
         'RI': 'Rhode Island',
         'SC': 'South Carolina',
         'SD': 'South Dakota',
         'TN': 'Tennessee',
         'TX': 'Texas',
         'UT': 'Utah',
         'VA': 'Virginia',
         'VI': 'Virgin Islands',
         'VT': 'Vermont',
         'WA': 'Washington',
         'WI': 'Wisconsin',
         'WV': 'West Virginia',
         'WY': 'Wyoming',
         'AB': 'Alberta',
          'BC': 'British Columbia',
          'MB': 'Manitoba',
          'NB': 'New Brunswick',
          'NL': 'Newfoundland and Labrador',
          'NS': 'Nova Scotia',
          'NT': 'Northwest Territories',
          'NU': 'Nunavut',
          'ON': 'Ontario',
          'PE': 'Prince Edward Island',
          'QC': 'Quebec',
          'SK': 'Saskatchewan',
          'YT': 'Yukon',
         "\'cause": 'because',
         '1st': 'first',
         '2nd': 'second',
         '3rd': 'third',
         '4th': 'fourth',
         "ain't": 'are not',
         'aint': 'are not',
         "aren't": 'are not',
         'arent': 'are not',
         'b/c': 'because',
         'bc': 'because',
         "can't": 'can not',
         "can't've": 'can not have',
         'cant': 'can not',
         'cause': 'because',
         "could've": 'could have',
         "couldn't": 'could not',
         "couldn't've": 'could not have',
         'couldnt': 'could not',
         'couldve': 'could have',
         "didn't": 'did not',
         'didnt': 'did not',
         "doesn't": 'does not',
         'doesnt': 'does not',
         "don't": 'do not',
         'dont': 'do not',
         "hadn't": 'had not',
         "hadn't've": 'had not have',
         'hadnt': 'had not',
         "hasn't": 'has not',
         'hasnt': 'has not',
         "haven't": 'have not',
         'havent': 'have not',
         "he'd": 'he would',
         "he'd've": 'he would have',
         "he'll": 'he will',
         "he'll've": 'he will have',
         "he's": 'he is',
         'hes': 'he is',
         "how'd": 'how did',
         "how'd'y": 'how do you',
         "how'll": 'how will',
         "how's": 'how is',
         'howd': 'how did',
         'howdy': 'how do you',
         'howll': 'how will',
         'hows': 'how is',
         "i'd": 'i would',
         "i'd've": 'i would have',
         "i'll": 'i will',
         "i'll've": 'i will have',
         "i'm": 'i am',
         "i've": 'i have',
         'id': 'i would',
         'ida': 'i would have',
         'im': 'i am',
         "isn't": 'is not',
         'isnt': 'is not',
         "it'd": 'it had',
         "it'd've": 'it would have',
         "it'll": 'it will',
         "it'll've": 'it will have',
         "it's": 'it is',
         'itd': 'it had',
         'itll': 'it will',
         'its': 'it is',
         'ive': 'i have',
         "let's": 'let us',
         'lets': 'let us',
         "ma'am": 'madam',
         'maam': 'madam',
         "mayn't": 'may not',
         "might've": 'might have',
         'mighta': 'might have',
         "mightn't": 'might not',
         "mightn't've": 'might not have',
         'mightnt': 'might not',
         'mightve': 'might have',
         "must've": 'must have',
         'musta': 'must have',
         "mustn't": 'must not',
         "mustn't've": 'must not have',
         'mustnt': 'must not',
         'mustve': 'must have',
         "needn't": 'need not',
         "needn't've": 'need not have',
         'neednt': 'need not',
         "o'clock": 'of the clock',
         'oclock': 'of the clock',
         "oughtn't": 'ought not',
         "oughtn't've": 'ought not have',
         "sha'n't": 'shall not',
         "shan't": 'shall not',
         "shan't've": 'shall not have',
         "she'd": 'she would',
         "she'd've": 'she would have',
         "she'll": 'she will',
         "she'll've": 'she will have',
         "she's": 'she is',
         'shes': 'she is',
         "should've": 'should have',
         'shoulda': 'should have',
         "shouldn't": 'should not',
         "shouldn't've": 'should not have',
         'shouldnt': 'should not',
         'shouldve': 'should have',
         "so'd": 'so did',
         "so's": 'so is',
         "so've": 'so have',
         "that'd": 'that would',
         "that'd've": 'that would have',
         "that's": 'that is',
         'thatd': 'that would',
         'thats': 'that is',
         "there'd": 'there had',
         "there'd've": 'there would have',
         "there's": 'there is',
         'thered': 'there had',
         'theres': 'there is',
         "they'd": 'they would',
         "they'd've": 'they would have',
         "they'll": 'they will',
         "they'll've": 'they will have',
         "they're": 'they are',
         "they've": 'they have',
         'theyd': 'they would',
         'theyda': 'they would have',
         'theyll': 'they will',
         'theyre': 'they are',
         'theyve': 'they have',
         "to've": 'to have',
         "wasn't": 'was not',
         'wasnt': 'was not',
         "we'd": 'we had',
         "we'd've": 'we would have',
         "we'll": 'we will',
         "we'll've": 'we will have',
         "we're": 'we are',
         "we've": 'we have',
         "weren't": 'were not',
         'werent': 'were not',
         'weve': 'we have',
         "what'll": 'what will',
         "what'll've": 'what will have',
         "what're": 'what are',
         "what's": 'what is',
         "what've": 'what have',
         'whatll': 'what will',
         'whatllve': 'what will have',
         'whatre': 'what are',
         'whats': 'what is',
         'whatve': 'what have',
         "when's": 'when is',
         "when've": 'when have',
         'whens': 'when is',
         'whenve': 'when have',
         "where'd": 'where did',
         "where's": 'where is',
         "where've": 'where have',
         'whered': 'where did',
         'whereve': 'where have',
         'whers': 'where is',
         "who'll": 'who will',
         "who'll've": 'who will have',
         "who's": 'who is',
         "who've": 'who have',
         'wholl': 'who will',
         'whollve': 'who will have',
         'whos': 'who is',
         'whove': 'who have',
         "why's": 'why is',
         "why've": 'why have',
         'whys': 'why is',
         'whyve': 'why have',
         "will've": 'will have',
         'willve': 'will have',
         "won't": 'will not',
         "won't've": 'will not have',
         'wont': 'will not',
         'wontve': 'will not have',
         "would've": 'would have',
         "wouldn't": 'would not',
         "wouldn't've": 'would not have',
         'wouldnt': 'would not',
         'wouldntve': 'would not have',
         'wouldve': 'would have',
         "y'all": 'you all',
         "y'all'd": 'you all would',
         "y'all'd've": 'you all would have',
         "y'all're": 'you all are',
         "y'all've": 'you all have',
         "y'alls": 'you alls',
         'yall': 'you all',
         'yalld': 'you all would',
         'yalldve': 'you all would have',
         'yallre': 'you all are',
         'yalls': 'you alls',
         'yallve': 'you all have',
         "you'd": 'you had',
         "you'd've": 'you would have',
         "you'da": 'you would have',
         "you'll": 'you you will',
         "you'll've": 'you you will have',
         "you're": 'you are',
         "you've": 'you have',
         'youd': 'you had',
         'youda': 'you would have',
         'youdve': 'you would have',
         'youll': 'you you will',
         'youllve': 'you you will have',
         'youre': 'you are',
         'youve': 'you have',     
         '2f4u': 'too fast for you',
         '4yeo': 'for your eyes only',
         'aamof': 'as a matter of fact',
         'ack': 'acknowledgment',
         'afaik': 'as far as i know',
         'afair': 'as far as i recall',
         'afk': 'away from keyboard',
         'aka': 'also known as',
         'ama': 'ask me anything',
         'b2k btk': 'back to keyboard',
         'bff': 'best friends forever',
         'brah': 'bro',
         'brb': 'be right back',
         'bs': 'bullshit',
         'btt': 'back to topic',
         'btw': 'by the way',
         'c&p': 'copy and paste',
         'cu': 'see you',
         'cys': 'check your settings',
         'dgaf': 'don’t give a fuck',
         'dgmw': 'do not get me wrong',
         'diy': 'do it yourself',
         'dtf': 'down to fuck',
         'eobd': 'end of business day',
         'eod': 'end of discussion',
         'eom': 'end of message',
         'eot': 'end of transmission',
         'fack': 'full acknowledge',
         'faq': 'frequently asked questions',
         'fb': 'facebook',
         'fbf': 'flashback friday',
         'fbo': 'facebook official',
         'ffs': 'for fuck sake',
         'fka': 'formerly known as',
         'fomo': 'fear of missing out',
         'ftw': 'for the win',
         'fu': 'fuck you',
         'fvck': 'fuck',
         'fwiw': "for what it's worth",
         'fyeo': 'for your eyes only',
         'fyi': 'for your information',
         'gg': 'good game',
         'gtfo': 'get the fuck out',
         'gtg': 'got to go',
         'hbd': 'happy birthday',
         'hf': 'have fun',
         'hmb': 'hit me back',
         'hmu': 'hit me up',
         'hth': 'hope this helps',
         'hwy': 'highway',
         'icymi': 'in case you missed it',
         'idc': 'i don’t care',
         'idk': 'i do not know',
         'iirc': 'if i recall',
         'ikr': 'i know right',
         'ily': 'i love you',
         'imho': 'in my humble opinion',
         'imnsho': 'in my not so honest opinion',
         'imo': 'in my opinion',
         'iow': 'in other words',
         'irl': 'in real life',
         'itt': 'in this thread',
         'jfyi': 'just for your information',
         'jk': 'just kidding',
         'lmk': 'let me know',
         'lms': 'like my status',
         'lol': 'laughing out loud',
         'mcm': 'mancrush monday',
         'mmw': 'mark my words',
         'n/a': 'not applicable',
         'n00b': 'newbie',
         'nm': 'not much',
         'nntr': 'no need to reply',
         'noob': 'newbie',
         'noyb': 'none of your business',
         'nrn': 'no reply necessary',
         'nsfw': 'not safe for work',
         'nvm': 'nevermind',
         'omg': 'oh my god',
         'omw': 'on my way',
         'ootd': 'outfit of the day',
         'op': 'original post',
         'ot': 'off topic',
         'otoh': 'on the other hand',
         'pebkac': 'problem exists between keyboard and chair',
         'potd': 'photo of the day',
         'pov': 'point of view',
         'ppl': 'people',
         'ptfo': 'passed the fuck out',
         'r': 'are',
         'rofl': 'rolling on the floor laughing',
         'roflmao': 'rolling on floor laughing my ass off',
         'rotfl': 'rolling on the floor laughing',
         'rsvp': 'repondez sil vous plait',
         'rt': 'retweet',
         'rtfm': 'read the fine manual',
         'scnr': 'sorry could not resist',
         'sflr': 'sorry for late reply',
         'sfw': 'safe for work',
         'smfh': 'shaking my fucking head',
         'spoc': 'single point of contact',
         'stfu': 'shut the fuck up',
         'sup': 'what is up',
         'tba': 'to be announced',
         'tbc': 'to be continued',
         'tbh': 'to be honest',
         'tbt': 'throwback thursday',
         'tgif': 'thanks god it is friday',
         'thx': 'thanks',
         'tia': 'thanks in advance',
         'tmi': 'too much information',
         'tnx': 'thanks',
         'tq': 'thank you',
         'ttyl': 'talk to you later',
         'ttyn': 'talk to you never',
         'ttys': 'talk to you soon',
         'txt': 'text',
         'tyt': 'take your time',
         'tyvm': 'thank you very much',
         'u': 'you',
         'ur': 'your',
         'w00t': 'woot',
         'wcw': 'womancrush wednesday',
         'wfm': 'works for me',
         'wrt': 'with regard to',
         'wtf': 'what the fuck',
         'wth': 'what the hell',
         'yam': 'yet another meeting',
         'ymmd': 'you made my day',
         'ymmv': 'your mileage may vary',
         'yolo': 'you only live once',
}
    c_re = re.compile('|'.join('(\b%s\b)' % re.escape(s) for s in word_expansion_dict.keys()), re.IGNORECASE)
    def replace(match):
        expansion =  f" {word_expansion_dict[match.group(0)]}"
        return expansion
    text = c_re.sub(replace, text.lower())
    return text

    # function to expand contractions, remove urls and characters before tokenization processing
    def denoise_text(text):
        new_text = re.sub(r"\S*https?:\S*",  r"", text.lower())
        new_text_contractions = expand_abbreviations_contractions(new_text)
        x = re.sub('#cafire', 'california fire', new_text_contractions)
        x = re.sub('#calfires', 'california fires', x)
        x = re.sub('#calwildfires', 'california wildfires', x)
        x = re.sub('#cadrought', 'california drought', x)
        new_text_punct = re.sub(r"[^\w\s@]",  r"", x)
        new_text_chars = re.sub('[^\u0000-\u007f]', '',  new_text_punct)
        strip_text = new_text_chars.strip()
        #remove_hashtags_text = re.sub('#\w+', '',  strip_text)
        return strip_text 
    
# tokenization & lemmatization function returns tokens    
    def lemmatize_text(text):
        tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
        lemmatizer = WordNetLemmatizer() 
        return [lemmatizer.lemmatize(w, pos='v') for w in tokenizer.tokenize(text)]

# tokenization & stemmer function returns tokens
    def stem_text(text):
        tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
        stemmer = PorterStemmer()
        return [stemmer.stem(w) for w in tokenizer.tokenize(text)]

    def replace_numbers(tokens):
# replace integers with string formatted words for numbers
        dig2word = inflect.engine()
        new_tokens = []
        for word in tokens:
            if word.isdigit():
                new_word = dig2word.number_to_words(word)
                new_tokens.append(new_word)
            else:
                new_tokens.append(word)
        return new_tokens
    
    def remove_non_ascii(tokens):
# remove non ascii characters from text
        new_tokens = []
        for word in tokens:
            new_token = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            new_tokens.append(new_token)
        return new_tokens
    
# remove stopwords   
    def remove_stopwords(tokens):
        stop_list = stopwords.words('english')  
        new_tokens = []
        for word in tokens:
            if word not in stop_list:
                new_tokens.append(word)
        return new_tokens
  
 
    def norm_text(tokens):
        words = replace_numbers(tokens)
        #tokens = remove_stopwords(words)
        words = remove_non_ascii(words)
        return words
    

    def process_text(text):
        clean_text = denoise_text(text)
        lem_text = lemmatize_text(clean_text)
        text = ' '.join([x for x in norm_text(lem_text)])
        text = re.sub(r"-",  r" ", text)
        return text
    
    new_df = [process_text(x) for x in df]

    return new_df 

In [285]:
def extract_hashtags(tweets):
    hashtag_list = []
    tweets = list(tweets)
    def denoise_text(text):
        new_text = re.sub(r"\S*https?:\S*",  r"", text.lower())
        new_text_punct = re.sub(r"[^\w\s]",  r"", new_text)
        new_text_chars = re.sub('[^\u0000-\u007f]', '',  new_text_punct)
        new_text_ = re.sub('_', '',  new_text_punct)
        return new_text_
    def replace_numbers(tokens):
# replace integers with string formatted words for numbers
        dig2word = inflect.engine()
        new_tokens = []
        for word in tokens:
            if word.isdigit():
                new_word = dig2word.number_to_words(word)
                new_tokens.append(new_word)
            else:
                new_tokens.append(word)
        return new_tokens
    # splitting the text into words
    for tweet in tweets:
        for x in tweet.split():
            if x.startswith('#') == True:
                clean_text = denoise_text(x)
                cleaner_text = replace_numbers(clean_text)
                hashtag_list.append(''.join(cleaner_text))
    return hashtag_list

In [21]:
def denoise_location(df):
    import re
def word_expansion(text):
    word_expansion_dict = {
         'AK': 'Alaska',
         'AL': 'Alabama',
         'AR': 'Arkansas',
         'AZ': 'Arizona',
         'CA': 'California',
         'CO': 'Colorado',
         'CT': 'Connecticut',
         'DC': 'District of Columbia',
         'DE': 'Delaware',
         'FL': 'Florida',
         'GA': 'Georgia',
         'GU': 'Guam',
         'HI': 'Hawaii',
         'IA': 'Iowa',
         'ID': 'Idaho',
         'IL': 'Illinois',
         'IN': 'Indiana',
         'KS': 'Kansas',
         'KY': 'Kentucky',
         'LA': 'Louisiana',
         'MA': 'Massachusetts',
         'MD': 'Maryland',
         'ME': 'Maine',
         'MI': 'Michigan',
         'MN': 'Minnesota',
         'MO': 'Missouri',
         'MP': 'Northern Mariana Islands',
         'MS': 'Mississippi',
         'MT': 'Montana',
         'NC': 'North Carolina',
         'ND': 'North Dakota',
         'NE': 'Nebraska',
         'NH': 'New Hampshire',
         'NJ': 'New Jersey',
         'NM': 'New Mexico',
         'NV': 'Nevada',
         'NY': 'New York',
         'OH': 'Ohio',
         'OK': 'Oklahoma',
         'OR': 'Oregon',
         'PA': 'Pennsylvania',
         'PR': 'Puerto Rico',
         'PW': 'Palau',
         'RI': 'Rhode Island',
         'SC': 'South Carolina',
         'SD': 'South Dakota',
         'TN': 'Tennessee',
         'TX': 'Texas',
         'UT': 'Utah',
         'VA': 'Virginia',
         'VI': 'Virgin Islands',
         'VT': 'Vermont',
         'WA': 'Washington',
         'WI': 'Wisconsin',
         'WV': 'West Virginia',
         'WY': 'Wyoming',
         'AB': 'Alberta',
          'BC': 'British Columbia',
          'MB': 'Manitoba',
          'NB': 'New Brunswick',
          'NL': 'Newfoundland and Labrador',
          'NS': 'Nova Scotia',
          'NT': 'Northwest Territories',
          'NU': 'Nunavut',
          'ON': 'Ontario',
          'PE': 'Prince Edward Island',
          'QC': 'Quebec',
          'SK': 'Saskatchewan',
          'YT': 'Yukon',
         "\'cause": 'because',
         '1st': 'first',
         '2nd': 'second',
         '3rd': 'third',
         '4th': 'fourth',
         "ain't": 'are not',
         'aint': 'are not',
         "aren't": 'are not',
         'arent': 'are not',
         'b/c': 'because',
         'bc': 'because',
         "can't": 'can not',
         "can't've": 'can not have',
         'cant': 'can not',
         'cause': 'because',
         "could've": 'could have',
         "couldn't": 'could not',
         "couldn't've": 'could not have',
         'couldnt': 'could not',
         'couldve': 'could have',
         "didn't": 'did not',
         'didnt': 'did not',
         "doesn't": 'does not',
         'doesnt': 'does not',
         "don't": 'do not',
         'dont': 'do not',
         "hadn't": 'had not',
         "hadn't've": 'had not have',
         'hadnt': 'had not',
         "hasn't": 'has not',
         'hasnt': 'has not',
         "haven't": 'have not',
         'havent': 'have not',
         "he'd": 'he would',
         "he'd've": 'he would have',
         "he'll": 'he will',
         "he'll've": 'he will have',
         "he's": 'he is',
         'hes': 'he is',
         "how'd": 'how did',
         "how'd'y": 'how do you',
         "how'll": 'how will',
         "how's": 'how is',
         'howd': 'how did',
         'howdy': 'how do you',
         'howll': 'how will',
         'hows': 'how is',
         "i'd": 'i would',
         "i'd've": 'i would have',
         "i'll": 'i will',
         "i'll've": 'i will have',
         "i'm": 'i am',
         "i've": 'i have',
         'id': 'i would',
         'ida': 'i would have',
         'im': 'i am',
         "isn't": 'is not',
         'isnt': 'is not',
         "it'd": 'it had',
         "it'd've": 'it would have',
         "it'll": 'it will',
         "it'll've": 'it will have',
         "it's": 'it is',
         'itd': 'it had',
         'itll': 'it will',
         'its': 'it is',
         'ive': 'i have',
         "let's": 'let us',
         'lets': 'let us',
         "ma'am": 'madam',
         'maam': 'madam',
         "mayn't": 'may not',
         "might've": 'might have',
         'mighta': 'might have',
         "mightn't": 'might not',
         "mightn't've": 'might not have',
         'mightnt': 'might not',
         'mightve': 'might have',
         "must've": 'must have',
         'musta': 'must have',
         "mustn't": 'must not',
         "mustn't've": 'must not have',
         'mustnt': 'must not',
         'mustve': 'must have',
         "needn't": 'need not',
         "needn't've": 'need not have',
         'neednt': 'need not',
         "o'clock": 'of the clock',
         'oclock': 'of the clock',
         "oughtn't": 'ought not',
         "oughtn't've": 'ought not have',
         "sha'n't": 'shall not',
         "shan't": 'shall not',
         "shan't've": 'shall not have',
         "she'd": 'she would',
         "she'd've": 'she would have',
         "she'll": 'she will',
         "she'll've": 'she will have',
         "she's": 'she is',
         'shes': 'she is',
         "should've": 'should have',
         'shoulda': 'should have',
         "shouldn't": 'should not',
         "shouldn't've": 'should not have',
         'shouldnt': 'should not',
         'shouldve': 'should have',
         "so'd": 'so did',
         "so's": 'so is',
         "so've": 'so have',
         "that'd": 'that would',
         "that'd've": 'that would have',
         "that's": 'that is',
         'thatd': 'that would',
         'thats': 'that is',
         "there'd": 'there had',
         "there'd've": 'there would have',
         "there's": 'there is',
         'thered': 'there had',
         'theres': 'there is',
         "they'd": 'they would',
         "they'd've": 'they would have',
         "they'll": 'they will',
         "they'll've": 'they will have',
         "they're": 'they are',
         "they've": 'they have',
         'theyd': 'they would',
         'theyda': 'they would have',
         'theyll': 'they will',
         'theyre': 'they are',
         'theyve': 'they have',
         "to've": 'to have',
         "wasn't": 'was not',
         'wasnt': 'was not',
         "we'd": 'we had',
         "we'd've": 'we would have',
         "we'll": 'we will',
         "we'll've": 'we will have',
         "we're": 'we are',
         "we've": 'we have',
         "weren't": 'were not',
         'werent': 'were not',
         'weve': 'we have',
         "what'll": 'what will',
         "what'll've": 'what will have',
         "what're": 'what are',
         "what's": 'what is',
         "what've": 'what have',
         'whatll': 'what will',
         'whatllve': 'what will have',
         'whatre': 'what are',
         'whats': 'what is',
         'whatve': 'what have',
         "when's": 'when is',
         "when've": 'when have',
         'whens': 'when is',
         'whenve': 'when have',
         "where'd": 'where did',
         "where's": 'where is',
         "where've": 'where have',
         'whered': 'where did',
         'whereve': 'where have',
         'whers': 'where is',
         "who'll": 'who will',
         "who'll've": 'who will have',
         "who's": 'who is',
         "who've": 'who have',
         'wholl': 'who will',
         'whollve': 'who will have',
         'whos': 'who is',
         'whove': 'who have',
         "why's": 'why is',
         "why've": 'why have',
         'whys': 'why is',
         'whyve': 'why have',
         "will've": 'will have',
         'willve': 'will have',
         "won't": 'will not',
         "won't've": 'will not have',
         'wont': 'will not',
         'wontve': 'will not have',
         "would've": 'would have',
         "wouldn't": 'would not',
         "wouldn't've": 'would not have',
         'wouldnt': 'would not',
         'wouldntve': 'would not have',
         'wouldve': 'would have',
         "y'all": 'you all',
         "y'all'd": 'you all would',
         "y'all'd've": 'you all would have',
         "y'all're": 'you all are',
         "y'all've": 'you all have',
         "y'alls": 'you alls',
         'yall': 'you all',
         'yalld': 'you all would',
         'yalldve': 'you all would have',
         'yallre': 'you all are',
         'yalls': 'you alls',
         'yallve': 'you all have',
         "you'd": 'you had',
         "you'd've": 'you would have',
         "you'da": 'you would have',
         "you'll": 'you you will',
         "you'll've": 'you you will have',
         "you're": 'you are',
         "you've": 'you have',
         'youd': 'you had',
         'youda': 'you would have',
         'youdve': 'you would have',
         'youll': 'you you will',
         'youllve': 'you you will have',
         'youre': 'you are',
         'youve': 'you have',     
         '2f4u': 'too fast for you',
         '4yeo': 'for your eyes only',
         'aamof': 'as a matter of fact',
         'ack': 'acknowledgment',
         'afaik': 'as far as i know',
         'afair': 'as far as i recall',
         'afk': 'away from keyboard',
         'aka': 'also known as',
         'ama': 'ask me anything',
         'b2k btk': 'back to keyboard',
         'bff': 'best friends forever',
         'brah': 'bro',
         'brb': 'be right back',
         'bs': 'bullshit',
         'btt': 'back to topic',
         'btw': 'by the way',
         'c&p': 'copy and paste',
         'cu': 'see you',
         'cys': 'check your settings',
         'dgaf': 'don’t give a fuck',
         'dgmw': 'do not get me wrong',
         'diy': 'do it yourself',
         'dtf': 'down to fuck',
         'eobd': 'end of business day',
         'eod': 'end of discussion',
         'eom': 'end of message',
         'eot': 'end of transmission',
         'fack': 'full acknowledge',
         'faq': 'frequently asked questions',
         'fb': 'facebook',
         'fbf': 'flashback friday',
         'fbo': 'facebook official',
         'ffs': 'for fuck sake',
         'fka': 'formerly known as',
         'fomo': 'fear of missing out',
         'ftw': 'for the win',
         'fu': 'fuck you',
         'fvck': 'fuck',
         'fwiw': "for what it's worth",
         'fyeo': 'for your eyes only',
         'fyi': 'for your information',
         'gg': 'good game',
         'gtfo': 'get the fuck out',
         'gtg': 'got to go',
         'hbd': 'happy birthday',
         'hf': 'have fun',
         'hmb': 'hit me back',
         'hmu': 'hit me up',
         'hth': 'hope this helps',
         'hwy': 'highway',
         'icymi': 'in case you missed it',
         'idc': 'i don’t care',
         'idk': 'i do not know',
         'iirc': 'if i recall',
         'ikr': 'i know right',
         'ily': 'i love you',
         'imho': 'in my humble opinion',
         'imnsho': 'in my not so honest opinion',
         'imo': 'in my opinion',
         'iow': 'in other words',
         'irl': 'in real life',
         'itt': 'in this thread',
         'jfyi': 'just for your information',
         'jk': 'just kidding',
         'lmk': 'let me know',
         'lms': 'like my status',
         'lol': 'laughing out loud',
         'mcm': 'mancrush monday',
         'mmw': 'mark my words',
         'n/a': 'not applicable',
         'n00b': 'newbie',
         'nm': 'not much',
         'nntr': 'no need to reply',
         'noob': 'newbie',
         'noyb': 'none of your business',
         'nrn': 'no reply necessary',
         'nsfw': 'not safe for work',
         'nvm': 'nevermind',
         'omg': 'oh my god',
         'omw': 'on my way',
         'ootd': 'outfit of the day',
         'op': 'original post',
         'ot': 'off topic',
         'otoh': 'on the other hand',
         'pebkac': 'problem exists between keyboard and chair',
         'potd': 'photo of the day',
         'pov': 'point of view',
         'ppl': 'people',
         'ptfo': 'passed the fuck out',
         'r': 'are',
         'rofl': 'rolling on the floor laughing',
         'roflmao': 'rolling on floor laughing my ass off',
         'rotfl': 'rolling on the floor laughing',
         'rsvp': 'repondez sil vous plait',
         'rt': 'retweet',
         'rtfm': 'read the fine manual',
         'scnr': 'sorry could not resist',
         'sflr': 'sorry for late reply',
         'sfw': 'safe for work',
         'smfh': 'shaking my fucking head',
         'spoc': 'single point of contact',
         'stfu': 'shut the fuck up',
         'sup': 'what is up',
         'tba': 'to be announced',
         'tbc': 'to be continued',
         'tbh': 'to be honest',
         'tbt': 'throwback thursday',
         'tgif': 'thanks god it is friday',
         'thx': 'thanks',
         'tia': 'thanks in advance',
         'tmi': 'too much information',
         'tnx': 'thanks',
         'tq': 'thank you',
         'ttyl': 'talk to you later',
         'ttyn': 'talk to you never',
         'ttys': 'talk to you soon',
         'txt': 'text',
         'tyt': 'take your time',
         'tyvm': 'thank you very much',
         'u': 'you',
         'ur': 'your',
         'w00t': 'woot',
         'wcw': 'womancrush wednesday',
         'wfm': 'works for me',
         'wrt': 'with regard to',
         'wtf': 'what the fuck',
         'wth': 'what the hell',
         'yam': 'yet another meeting',
         'ymmd': 'you made my day',
         'ymmv': 'your mileage may vary',
         'yolo': 'you only live once',
}
    c_re = re.compile('|'.join('(\b%s\b)' % (re.escape(s) for s in word_expansions.keys()), re.IGNORECASE))
    def replace(match):
        expansion =  f" {contractions_dict[match.group(0)]}"
        return expansion
    text = c_re.sub(replace, text.lower())
    return text
    # tokenization & lemmatization function returns tokens    
    def lemmatize_text(text):
        tokenizer = TweetTokenizer(strip_handles=True)
        lemmatizer = WordNetLemmatizer() 
        return [lemmatizer.lemmatize(w, pos='n') for w in tokenizer.tokenize(text)]

    
    def remove_non_ascii(tokens):
# remove non ascii characters from text
        new_tokens = []
        for word in tokens:
            new_token = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            new_tokens.append(new_token)
        return new_tokens
    
    def norm_text(tokens):
        words = remove_non_ascii(tokens)
        return words
    
    def denoise_location_text(text):
        text = str(text)
        new_text = re.sub(r"\S*https?:\S*",  r"", text.lower())
        new_string = re.sub(r"[^\w\s]",  r" ", new_text)
        new_string = re.sub(r"\d",  r"", new_string)
        unicode_string = re.sub('[^\u0000-\u007f]', '',  new_string)
        new_text_contractions = expand_abbreviations_contractions(unicode_string)
        clean_text = re.sub(r"est september   ",  r"", new_text_contractions)
        lem_text = lemmatize_text(clean_text)
        text = ' '.join([x for x in norm_text(lem_text)])
        text = re.sub(r"-",  r" ", text)
        return text

    
    new_df = [denoise_location_text(x) for x in df]
    return new_df 



In [32]:
#hashtags = extract_hashtags(df.text)
df['tweets'] = tweet_preprocess((df.text.astype(str).replace({r"([#[A-Z][a-z])": r" \1"}, regex=True)))
df['keyword'] = tweet_preprocess(df.keyword.astype(str).replace({r"%20" : r" "}, regex=True))

NameError: name 'word_expansion_dict' is not defined

In [333]:
df[:50]

,id,keyword,location,text,target,tweets
0,1,nan,NaN,Our Deeds are the Reason of this #earthquake M...,1,our deeds be the reason of this earthquake may...
1,4,nan,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,5,nan,NaN,All residents asked to 'shelter in place' are ...,1,all residents ask to shelter in place be be no...
3,6,nan,NaN,"13,000 people receive #wildfires evacuation or...",1,thirteen thousand people receive wildfires eva...
4,7,nan,NaN,Just got sent this photo from Ruby #Alaska as ...,1,just get send this photo from ruby alaska as s...
5,8,nan,NaN,#RockyFire Update => California Hwy. 20 closed...,1,rocky fire update california highway twenty cl...
6,10,nan,NaN,#flood #disaster Heavy rain causes flash flood...,1,flood disaster heavy rain becauses flash flood...
7,13,nan,NaN,I'm on top of the hill and I can see a fire in...,1,im on top of the hill and i can see a fire in ...
8,14,nan,NaN,There's an emergency evacuation happening now ...,1,there be an emergency evacuation happen now in...
9,15,nan,NaN,I'm afraid that the tornado is coming to our a...,1,im afraid that the tornado be come to our area


In [39]:
df['clean_location'] = denoise_location(df.location)
df['clean_location'] = clean_loc

In [24]:
train_X = df.filter(['tweets','clean_location','keyword'], axis=1)
train_y = df.filter(['target'], axis=1)

In [43]:
all_text = df.tweets + " " + df.clean_location + " " + df.keyword
all_text

0       our deeds be the reason of this earthquake may...
1           forest fire near la ronge sask canada nan nan
2       all residents ask to shelter in place be be no...
3       thirteen thousand people receive wildfires eva...
4       just get send this photo from ruby alaska as s...
                              ...                        
7604    world news fall powerlines on glink tram you p...
7605    on the flip side i be at walmart and there be ...
7606    suicide bomber kill fifteen in saudi security ...
7608    two giant crane hold a bridge collapse into ne...
7612    the latest more home raze by northern californ...
Length: 7503, dtype: object

In [44]:
df['all_text'] = all_text
df

,id,keyword,location,text,target,tweets,clean_location,all_text
0,1,nan,nan,Our Deeds are the Reason of this #earthqua...,1,our deeds be the reason of this earthquake may...,nan,our deeds be the reason of this earthquake may...
1,4,nan,nan,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada,nan,forest fire near la ronge sask canada nan nan
2,5,nan,nan,All residents asked to 'shelter in place' ar...,1,all residents ask to shelter in place be be no...,nan,all residents ask to shelter in place be be no...
3,6,nan,nan,"13,000 people receive #wildfires evacuation o...",1,thirteen thousand people receive wildfires eva...,nan,thirteen thousand people receive wildfires eva...
4,7,nan,nan,Just got sent this photo from Ruby # Alaska...,1,just get send this photo from ruby alaska as s...,nan,just get send this photo from ruby alaska as s...
...,...,...,...,...,...,...,...,...
7604,10863,nan,nan,# World News Fallen powerlines on G:link tr...,1,world news fall powerlines on glink tram you p...,nan,world news fall powerlines on glink tram you p...
7605,10864,nan,nan,on the flip side I'm at Walmart and there i...,1,on the flip side i be at walmart and there be ...,nan,on the flip side i be at walmart and there be ...
7606,10866,nan,nan,Suicide bomber kills 15 in Saudi security s...,1,suicide bomber kill fifteen in saudi security ...,nan,suicide bomber kill fifteen in saudi security ...
7608,10869,nan,nan,Two giant cranes holding a bridge collapse i...,1,two giant crane hold a bridge collapse into ne...,nan,two giant crane hold a bridge collapse into ne...


In [ ]:
pos_text_tags = df['tweets'].apply(lambda row: [nltk.pos_tag(row) for item in row])
pos_keyword_tags = df['keyword'].apply(lambda row: [nltk.pos_tag(row) for item in row])

In [ ]:
df.location[df.location != 'nan']

In [ ]:
df.keyword.unique()

In [ ]:
plt.figure(figsize=(11,11))
colors = ['lightblue', 'red']
expl = (0, 0.1)
df.target.value_counts().plot(kind='pie', legend=True, startangle=45, shadow=True, 
                             colors=colors, autopct='%1.1f%%')
plt.title('target count', fontsize=20)

In [ ]:
spell = SpellChecker()

In [ ]:
misspelled = spell.unknown(df.tweets)

In [ ]:
misspelled

In [ ]:
pip install gensim

In [ ]:
import gensim
from gensim import corpora

In [ ]:
corpus = df.tweets
nlp = Word2Vec(corpus, size=200,   
            window=6, min_count=1, sg=1, iter=40)
len(nlp.wv.vocab) # number of words in a dictionary

In [ ]:
import pkg_resources
from symspellpy.symspellpy import SymSpell

In [ ]:
hashtags[99:]

In [ ]:
df

In [ ]:
new_hash=[]
sym_spell = SymSpell(max_dictionary_edit_distance=1)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
for term in hashtags:
    if len(term)>1:
        result = sym_spell.word_segmentation(term)
        r = result.corrected_string
    else:
        r = ''
    new_hash.append(r)

In [ ]:
df[10:50]

In [ ]:
new_hash

In [ ]:
# Creating the dicti
word_dict = {} 

for row in df.tweets: 
    words = tokenizer.tokenize(row) 
    for word in words: 
        if word not in word_dict.keys(): 
            word_dict[word] = 1
        else: 
            word_dict[word] += 1
print(len(word_dict))
max(word_dict, key=word_dict.get)

In [ ]:
word_dict

In [ ]:
hlist=[]
for x in hashtags:
    x = x.str.replace('[^a-zA-Z]', '')
    hlist.append(x)